#**How to select best model**
1. train multiple model with different hyperparameter on train set.

2. evaluate each model's on validation set.

3. select best model.

4. train best model again with train set + validation set.

5. got final model.

6. evaluate with test set.

.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import random

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

print(f"TensorFlow Version: {tf.__version__}")

# --- 1. Load and Prepare Data (Using MNIST as a nice dataset) ---
print("--- 1. Loading and preparing the MNIST dataset ---")
(x_train_full, y_train_full), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize pixel values to be between 0 and 1
x_train_full = x_train_full.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Reshape data for CNN input (add channel dimension)
x_train_full = np.expand_dims(x_train_full, -1)
x_test = np.expand_dims(x_test, -1)

# Split full training data into training and validation sets
# Using 10,000 samples for validation
x_train, x_val = x_train_full[:-10000], x_train_full[-10000:]
y_train, y_val = y_train_full[:-10000], y_train_full[-10000:]

print(f"Training data shape: {x_train.shape}, labels shape: {y_train.shape}")
print(f"Validation data shape: {x_val.shape}, labels shape: {y_val.shape}")
print(f"Test data shape: {x_test.shape}, labels shape: {y_test.shape}")

# --- 2. Define the Model Architecture ---
def build_model(learning_rate=0.001):
    """
    Builds a simple Convolutional Neural Network (CNN) model.
    Args:
        learning_rate (float): The learning rate for the Adam optimizer.
    Returns:
        keras.Model: The compiled Keras model.
    """
    model = keras.Sequential([
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(10, activation="softmax"),
    ])
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

# --- 3. Train Multiple Models with Different Hyperparameters ---
print("\n--- 3. Training multiple models with different hyperparameters ---")

# Define hyperparameter combinations to test
hyperparameters = [
    {"learning_rate": 0.01, "batch_size": 32, "epochs": 5},
    {"learning_rate": 0.001, "batch_size": 32, "epochs": 5},
    {"learning_rate": 0.0005, "batch_size": 64, "epochs": 5},
    {"learning_rate": 0.001, "batch_size": 64, "epochs": 5},
]

models = []
histories = []
validation_accuracies = []

for i, params in enumerate(hyperparameters):
    print(f"\n--- Training Model {i+1}/{len(hyperparameters)} ---")
    print(f"Hyperparameters: {params}")

    # Build a new model for each hyperparameter set
    model = build_model(learning_rate=params["learning_rate"])

    # Train the model on the training set
    history = model.fit(x_train, y_train,
                        batch_size=params["batch_size"],
                        epochs=params["epochs"],
                        verbose=0) # Set verbose to 1 to see training progress per epoch

    models.append(model)
    histories.append(history)

    # --- 4. Evaluate Each Model on the Validation Set ---
    print(f"--- 4. Evaluating Model {i+1} on the validation set ---")
    val_loss, val_accuracy = model.evaluate(x_val, y_val, verbose=0)
    validation_accuracies.append(val_accuracy)
    print(f"Validation Accuracy: {val_accuracy:.4f}, Validation Loss: {val_loss:.4f}")

# --- 5. Select the Best Model ---
print("\n--- 5. Selecting the best model ---")
best_model_index = np.argmax(validation_accuracies)
best_val_accuracy = validation_accuracies[best_model_index]
best_params = hyperparameters[best_model_index]

print(f"Best Model Index: {best_model_index + 1}")
print(f"Best Validation Accuracy: {best_val_accuracy:.4f}")
print(f"Best Hyperparameters: {best_params}")

# --- 6. Train Best Model Again with Train Set + Validation Set ---
print("\n--- 6. Retraining the best model on the combined train and validation set ---")

# Combine training and validation sets
x_train_combined = np.concatenate((x_train, x_val), axis=0)
y_train_combined = np.concatenate((y_train, y_val), axis=0)

print(f"Combined training data shape: {x_train_combined.shape}, labels shape: {y_train_combined.shape}")

# Build a new model instance with the best hyperparameters
final_model = build_model(learning_rate=best_params["learning_rate"])

# Train the final model
# Using more epochs for the final training, as it's on a larger dataset
final_epochs = 10 # You can adjust this based on dataset size and complexity
print(f"Training final model for {final_epochs} epochs with batch size {best_params['batch_size']}")
final_history = final_model.fit(x_train_combined, y_train_combined,
                                batch_size=best_params["batch_size"],
                                epochs=final_epochs,
                                verbose=1) # Show verbose output for final training

# --- 7. Evaluate with Test Set ---
print("\n--- 7. Evaluating the final model on the test set ---")
test_loss, test_accuracy = final_model.evaluate(x_test, y_test, verbose=1)

print(f"\nFinal Model Test Accuracy: {test_accuracy:.4f}")
print(f"Final Model Test Loss: {test_loss:.4f}")

print("\n--- Workflow Complete ---")


TensorFlow Version: 2.18.0
--- 1. Loading and preparing the MNIST dataset ---
11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training data shape: (50000, 28, 28, 1), labels shape: (50000,)
Validation data shape: (10000, 28, 28, 1), labels shape: (10000,)
Test data shape: (10000, 28, 28, 1), labels shape: (10000,)

--- 3. Training multiple models with different hyperparameters ---

--- Training Model 1/4 ---
Hyperparameters: {'learning_rate': 0.01, 'batch_size': 32, 'epochs': 5}
--- 4. Evaluating Model 1 on the validation set ---
Validation Accuracy: 0.9894, Validation Loss: 0.0456

--- Training Model 2/4 ---
Hyperparameters: {'learning_rate': 0.001, 'batch_size': 32, 'epochs': 5}
--- 4. Evaluating Model 2 on the validation set ---
Validation Accuracy: 0.9906, Validation Loss: 0.0361

--- Training Model 3/4 ---
Hyperparameters: {'learning_rate': 0.0005, 'batch_size': 64, 'epochs': 5}
--- 4. Evaluating Model 3 on the validation set ---
Validation Accuracy: 0.9850, Validation Loss: 0.